# Atproto OAuth

This turned out to be a major pain in the butt to get to work with Deno. See
[this](https://github.com/bluesky-social/atproto/discussions/3303)

Start by patching `jwk-jose` from
https://github.com/bluesky-social/atproto/tree/main/packages/oauth/jwk-jose.
Basically copy what's there in atproto repo and use jose from denoland URL.
Also, patch this line to include algorithm explicitly (see
[this](https://github.com/panva/jose/discussions/648)):

```ts
return (this.#keyObj ||= await importJWK(this.jwk as JWK, "ES256"));
```

In [ ]:
// |export

import { JwtVerifyError } from "@atproto/jwk";
import {
  errors,
  exportJWK,
  generateKeyPair,
  type GenerateKeyPairOptions,
  type GenerateKeyPairResult,
  importJWK,
  importPKCS8,
  type JWK,
  jwtVerify,
  type JWTVerifyOptions,
  type KeyLike,
  SignJWT,
} from "https://deno.land/x/jose@v5.9.6/index.ts";

import {
  Jwk,
  JwkError,
  jwkValidator,
  JwtCreateError,
  JwtHeader,
  JwtPayload,
  Key,
  SignedJwt,
  VerifyOptions,
  VerifyPayload,
  VerifyResult,
} from "@atproto/jwk";

import { SimpleStore } from "@atproto-labs/simple-store";
import { InternalStateData, OAuthClient, Session } from "@atproto/oauth-client";
import { createHash, randomBytes } from "node:crypto";


export function either<T extends string | number | boolean>(
  a?: T,
  b?: T,
): T | undefined {
  if (a != null && b != null && a !== b) {
    throw new TypeError(`Expected "${b}", got "${a}"`);
  }
  return a ?? b ?? undefined;
}

const { JOSEError } = errors;

export type Importable = string | KeyLike | Jwk;

export type { GenerateKeyPairOptions, GenerateKeyPairResult };

export class JoseKey extends Key {
  #keyObj?: KeyLike | Uint8Array;

  protected async getKey() {
    try {
      // @ts-ignore readonly
      this.jwk.key_ops = ["sign"];
      // @ts-ignore readonly
      this.jwk.ext = true;
      return (this.#keyObj ||= await importJWK(this.jwk as JWK, "ES256"));
    } catch (cause) {
      throw new JwkError("Failed to import JWK", undefined, { cause });
    }
  }

  async createJwt(header: JwtHeader, payload: JwtPayload) {
    if (header.kid && header.kid !== this.kid) {
      throw new JwtCreateError(
        `Invalid "kid" (${header.kid}) used to sign with key "${this.kid}"`,
      );
    }

    if (!header.alg || !this.algorithms.includes(header.alg)) {
      throw new JwtCreateError(
        `Invalid "alg" (${header.alg}) used to sign with key "${this.kid}"`,
      );
    }

    const keyObj = await this.getKey();
    return new SignJWT(payload)
      .setProtectedHeader({ ...header, kid: this.kid })
      .sign(keyObj) as Promise<SignedJwt>;
  }

  async verifyJwt<
    P extends VerifyPayload = JwtPayload,
    C extends string = string,
  >(token: SignedJwt, options?: VerifyOptions<C>): Promise<VerifyResult<P, C>> {
    try {
      const keyObj = await this.getKey();
      const result = await jwtVerify(token, keyObj, {
        ...options,
        algorithms: this.algorithms,
      } as JWTVerifyOptions);

      return result as VerifyResult<P, C>;
    } catch (error) {
      if (error instanceof JOSEError) {
        throw new JwtVerifyError(error.message, error.code, { cause: error });
      } else {
        throw JwtVerifyError.from(error);
      }
    }
  }

  static async generateKeyPair(
    allowedAlgos: readonly string[] = ["ES256"],
    options?: GenerateKeyPairOptions,
  ) {
    if (!allowedAlgos.length) {
      throw new JwkError("No algorithms provided for key generation");
    }

    const errors: unknown[] = [];
    for (const alg of allowedAlgos) {
      try {
        return await generateKeyPair(alg, options);
      } catch (err) {
        errors.push(err);
      }
    }

    throw new JwkError("Failed to generate key pair", undefined, {
      cause: new AggregateError(errors, "None of the algorithms worked"),
    });
  }

  static async generate(
    allowedAlgos: string[] = ["ES256"],
    kid?: string,
    options?: Omit<GenerateKeyPairOptions, "extractable">,
  ) {
    const kp = await this.generateKeyPair(allowedAlgos, {
      ...options,
      extractable: true,
    });
    return this.fromImportable(kp.privateKey, kid);
  }

  static async fromImportable(
    input: Importable,
    kid?: string,
  ): Promise<JoseKey> {
    if (typeof input === "string") {
      // PKCS8
      if (input.startsWith("-----")) {
        // The "alg" is only needed in WebCrypto (NodeJS will be fine)
        return await this.fromPKCS8(input, "", kid);
      }

      // Jwk (string)
      if (input.startsWith("{")) {
        return this.fromJWK(input, kid);
      }

      throw new JwkError("Invalid input");
    }

    if (typeof input === "object") {
      // Jwk
      if ("kty" in input || "alg" in input) {
        return this.fromJWK(input, kid);
      }

      // KeyLike
      return this.fromKeyLike(input, kid);
    }

    throw new JwkError("Invalid input");
  }

  /**
   * @see {@link exportJWK}
   */
  static async fromKeyLike(
    keyLike: KeyLike | Uint8Array,
    kid?: string,
    alg?: string,
  ): Promise<JoseKey> {
    const jwk = await exportJWK(keyLike);
    if (alg) {
      if (!jwk.alg) jwk.alg = alg;
      else if (jwk.alg !== alg) throw new JwkError('Invalid "alg" in JWK');
    }
    // @ts-ignore cannot convert JWK to string
    return this.fromJWK(jwk, kid);
  }

  /**
   * @see {@link importPKCS8}
   */
  static async fromPKCS8(
    pem: string,
    alg: string,
    kid?: string,
  ): Promise<JoseKey> {
    const keyLike = await importPKCS8(pem, alg, { extractable: true });
    return this.fromKeyLike(keyLike, kid);
  }

  static fromJWK(
    input: string | Record<string, unknown>,
    inputKid?: string,
  ): JoseKey {
    const jwk = typeof input === "string" ? JSON.parse(input) : input;
    if (!jwk || typeof jwk !== "object") throw new JwkError("Invalid JWK");

    const kid = either(jwk.kid, inputKid);
    const use = jwk.use || "sig";

    return new JoseKey(jwkValidator.parse({ ...jwk, kid, use }));
  }
}

# Recreate dpop-store

This is from
https://github.com/bluesky-social/atproto/blob/main/packages/oauth/oauth-client-node/src/node-dpop-store.ts
to simplify key storage

In [ ]:
// |export

type ToDpopJwkValue<V extends { dpopKey: Key }> = Omit<V, "dpopKey"> & {
  dpopJwk: Jwk;
};

/**
 * Utility function that allows to simplify the store interface by exposing a
 * JWK (JSON) instead of a Key instance.
 */
export function toDpopKeyStore<K extends string, V extends { dpopKey: Key }>(
  store: SimpleStore<K, ToDpopJwkValue<V>>
): SimpleStore<K, V> {
  return {
    async set(sub: K, { dpopKey, ...data }: V) {
      const dpopJwk = dpopKey.privateJwk;
      if (!dpopJwk) throw new Error("Private DPoP JWK is missing.");

      await store.set(sub, { ...data, dpopJwk });
    },

    async get(sub: K) {
      const result = await store.get(sub);
      if (!result) return undefined;

      const { dpopJwk, ...data } = result;
      const dpopKey = await JoseKey.fromJWK(dpopJwk);
      return { ...data, dpopKey } as unknown as V;
    },

    del: store.del.bind(store),
    clear: store.clear?.bind(store),
  };
}

export type NodeSavedState = ToDpopJwkValue<InternalStateData>;
export type NodeSavedStateStore = SimpleStore<string, NodeSavedState>;

export type NodeSavedSession = ToDpopJwkValue<Session>;
export type NodeSavedSessionStore = SimpleStore<string, NodeSavedSession>;


In [ ]:
// |export

export { OAuthClient } from "@atproto/oauth-client";

// set this to the public URL of the app
const publicUrl =  Deno.env.get("PUBLIC_URL");

export const getOAuthClient = async () => {
  const kv = await Deno.openKv();

  return new OAuthClient({
    handleResolver: "https://api.bsky.app", // backend instances should use a DNS based resolver
    responseMode: "query",
    clientMetadata: {
      client_name: "AT Protocol Express App",
      client_id: publicUrl?.replace(/\/$/gi, "") + "/client-metadata.json",
      client_uri: publicUrl,
      redirect_uris: [`${publicUrl}/oauth/callback`],
      scope: "atproto transition:generic",
      grant_types: ["authorization_code", "refresh_token"],
      response_types: ["code"],
      application_type: "web",
      token_endpoint_auth_method: "none",
      dpop_bound_access_tokens: true,
    },

    stateStore: toDpopKeyStore({
      // A store for saving state data while the user is being redirected to the
      // authorization server.

      async set(key: string, internalState: NodeSavedState) {
        await kv.set(["atprotostate", key], JSON.stringify(internalState));
      },
      async get(key: string): Promise<NodeSavedState | undefined> {
        const v =
          (await kv.get<string | undefined>(["atprotostate", key])).value ||
          undefined;
        return v && JSON.parse(v);
      },
      async del(key: string) {
        await kv.delete(["atprotosession", key]);
      },
    }),

    sessionStore: toDpopKeyStore({
      // A store for saving session data.

      async set(sub: string, session: NodeSavedSession) {
        await kv.set(["atprotosession", sub], JSON.stringify(session));
      },
      async get(sub: string): Promise<NodeSavedSession | undefined> {
        const v =
          (await kv.get<string | undefined>(["atprotosession", sub])).value ||
          undefined;
        return v && JSON.parse(v);
      },
      async del(sub: string) {
        await kv.delete(["atprotosession", sub]);
      },
    }),

    runtimeImplementation: {
      // A runtime specific implementation of the crypto operations needed by the
      // OAuth client. See "@atproto/oauth-client-browser" for a browser specific
      // implementation. The following example is suitable for use in NodeJS.

      createKey(algs: string[]) {
        return JoseKey.generate(algs);
      },
      getRandomValues: randomBytes,
      digest(bytes: Uint8Array, algorithm: { name: string }) {
        return createHash(algorithm.name).update(bytes).digest();
      },
    },
  });
};  

[Module: null prototype] {
  AppViewHandleResolver: [class AppViewHandleResolver],
  AtprotoDohHandleResolver: [class AtprotoDohHandleResolver extends AtprotoHandleResolver],
  AtprotoHandleResolver: [class AtprotoHandleResolver],
  CLIENT_ASSERTION_TYPE_JWT_BEARER: "urn:ietf:params:oauth:client-assertion-type:jwt-bearer",
  CachedHandleResolver: [class CachedHandleResolver],
  DID_PLC_PREFIX: "did:plc:",
  DID_PREFIX: "did:",
  DID_WEB_PREFIX: "did:web:",
  DidCacheMemory: [class DidCacheMemory extends SimpleStoreMemory],
  DidError: [class DidError extends Error],
  DidPlcMethod: [class DidPlcMethod],
  DidResolverCached: [class DidResolverCached],
  DidResolverCommon: [class DidResolverCommon extends DidResolverBase],
  DidWebMethod: [class DidWebMethod],
  FetchError: [class FetchError extends Error],
  FetchRequestError: [class FetchRequestError extends FetchError],
  FetchResponseError: [class FetchResponseError extends FetchError],
  InvalidDidError: [class InvalidDidError exten